In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict, load_dataset
import torch
import random

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!unzip open.zip

In [22]:
df_train=pd.read_csv('train.csv')
df_train_copy=df_train.copy()

In [23]:
df_train

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [24]:
kind_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\CP2\augment_csv\df_kind_aug.csv')
polar_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\CP2\augment_csv\df_polar_aug.csv')
certainty_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\CP2\augment_csv\df_certainty_aug.csv')
tense_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\CP2\augment_csv\df_tense_aug.csv')


In [25]:
preprocess(df_train)
preprocess(kind_aug)
preprocess(polar_aug)
preprocess(certainty_aug)
preprocess(tense_aug)

In [ ]:
df_aug=pd.concat([df_train, kind_aug], axis=0).drop_duplicates(subset='문장', keep='first')
df_aug=pd.concat([df_aug, polar_aug], axis=0).drop_duplicates(subset='문장', keep='first')
df_aug=pd.concat([df_aug, tense_aug], axis=0).drop_duplicates(subset='문장', keep='first')
df_aug=pd.concat([df_aug, certainty_aug], axis=0).drop_duplicates(subset='문장', keep='first')
df_aug

In [28]:
df_aug=df_aug.drop([['ID', label]], axis=1)
df_aug.to_csv('df_aug.csv', index=False, encoding='utf-8-sig')

In [29]:
df_aug

,문장,유형,극성,시제,확실성
0,11포인트 금리 인상은 1년 이후 1년 만에 처음이다,사실형,긍정,현재,확실
1,이어 앞으로 전문가들과 함께 1주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...,사실형,긍정,과거,확실
2,정부가 고유가 대응을 위해 1월부터 연말까지 유류세 인하 폭을 1에서 1까지 확대한다,사실형,긍정,미래,확실
3,서울시는 올해 1월 즉시 견인 유예시간 1분을 제공하겠다고 밝혔지만 하루 만에 차도...,사실형,긍정,과거,확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다,사실형,긍정,현재,확실
...,...,...,...,...,...
20058,열린 a 투어 a 1타 차 단독 선두로 출발했지만 최종 라운드에서 무려 1타를 잃고...,NaN,NaN,NaN,확실
20059,열린 a 투어 a 한다 빅오픈에서 1타 단독 선두로 출발했지만 최종 라운드에서 무려...,NaN,NaN,NaN,확실
20060,앞서 열린 a 투어 a 한다 차 단독 선두로 출발했지만 최종 라운드에서 잃고 a 투...,NaN,NaN,NaN,확실
20061,앞서 열린 a 투어 a 단독 선두로 출발했지만 최종 라운드에서 무려 잃고 a 투어 ...,NaN,NaN,NaN,확실


In [11]:
kind_aug

,문장,유형
0,박 회장은 1년 1월 경부고속도로 대전공구 시험계장으로 발령을 받는다,사실형
1,국내 게임업계에서 신종 코로나바이러스 감염증코로나1 환자가 발생했다,사실형
2,코로나1는 인류에게 많은 변화를 요구하고 있다,사실형
3,또 자유공정혁신연대를 1대 정책 기조로 잡고 경제운용 중심축을 정부에서 민간으로 바...,사실형
4,젊은 층 외지인 수요가 몰리면서 집값이 하늘을 모르고 치솟았다,사실형
...,...,...
18719,이날 한밤중인 한때 접속 대기자가 넘겼고 오전 오후 1만1여명이 몰렸다,사실형
18720,이날 오전 한때 넘겼고 대기자가 1만명을 접속 한밤중인 오후 1만1여명이 몰렸다,사실형
18721,이날 오전 오후 접속 대기자가 넘겼고 한밤중인 한때 1시에도 1만1여명이 몰렸다,사실형
18722,이날 오전 한때 접속 대기자가 1만명을 1만1여명이 한밤중인 오후 넘겼고 몰렸다,사실형


In [276]:
pred_aug=pd.read_csv('pred_aug.csv')
non_aug=pd.read_csv('non_aug.csv')

In [22]:
df_train.drop_duplicates(subset=['문장', 'label'], inplace=True) #문장, 레이블 동시 중복 제거
df_train.reset_index(drop=True, inplace=True)
df_train['확실성'][14963]='확실'
df_train['label'][14963]='사실형-긍정-현재-확실' #맞게 변경

df_train.drop_duplicates(subset='문장', keep='last', inplace=True) #틀린 레이블을 가진 중복 문장 제거
df_train.reset_index(drop=True, inplace=True)

#df_train.describe(exclude='number')

In [155]:
def rs_rd_augmentation_preprocess(df, label, label_cat):
  
  new_df=df.copy()
  new_df = new_df[['문장', label]]
  new_df = new_df[new_df[label]==label_cat].reset_index(drop=True)
    
  if len(new_df) > 2000:
    new_df = new_df.sample(n=200)
    new_df.reset_index(drop=True, inplace=True)

  return new_df

In [531]:
df_train_true_aug = rs_rd_augmentation_preprocess(df_train, "유형", "사실형")
df_train_inf_aug = rs_rd_augmentation_preprocess(df_train, "유형", "추론형")
#df_train_pred_aug = rs_rd_augmentation_preprocess(df_train, "유형", "예측형")
df_train_conv_aug = rs_rd_augmentation_preprocess(df_train, "유형", "대화형")

df_train_pos_aug = rs_rd_augmentation_preprocess(df_train, "극성", "긍정")
df_train_neg_aug = rs_rd_augmentation_preprocess(df_train, "극성", "부정")
#df_train_non_aug = rs_rd_augmentation_preprocess(df_train, "극성", "미정")

df_train_pres_aug = rs_rd_augmentation_preprocess(df_train, "시제", "현재")
df_train_past_aug = rs_rd_augmentation_preprocess(df_train, "시제", "과거")
df_train_future_aug = rs_rd_augmentation_preprocess(df_train, "시제", "미래")

df_train_certain_aug = rs_rd_augmentation_preprocess(df_train, "확실성", "확실")
df_train_uncertain_aug = rs_rd_augmentation_preprocess(df_train, "확실성", "불확실")

In [2]:
import spacy
from spacy.lang.ko.examples import sentences 
nlp = spacy.load("ko_core_news_sm") #python -m spacy download ko_core_news_sm

In [3]:

doc = nlp("생각날 때 읽어")
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

생각날 때 읽어
생각날 VERB acl
때 NOUN dep
읽어 VERB ROOT


In [6]:
# open 불용어.txt
import locale

# 지역 설정 변경
locale.setlocale(locale.LC_COLLATE, 'ko_KR.UTF-8')

stop_words = {}

with open('불용어.txt', 'r') as f:
    for line in f:
        line = line.strip('\n')
        line = line.split(':')  # 줄 바꿈 문자 제거
        key = line[0] #key
        value = line[1].split(', ')  # value
        value.sort(key=locale.strxfrm)
        stop_words[key] = value  # 딕셔너리에 추가




In [278]:
# 특수문자, 괄호, 괄호 안 문장 제거 함수
def get_only_words(line):
  parseText= re.compile(r'\([^()]*\)').sub('',  line)
  parseText= re.compile('[^ \w]').sub('', parseText)

  return parseText

In [7]:
# stop_words_set
stop_words_lst=[]
for _, value in stop_words.items():
    stop_words_lst.extend(value)

stop_words_set=set(stop_words_lst)
stop_words_set.update(nlp.Defaults.stop_words)

In [18]:
def preprocess(df):
  df['문장']=df['문장'].map(lambda x: re.sub(r'[\d]+', '1', x)) #숫자를 1로 바꿈.
  df['문장']=df['문장'].map(lambda x: re.sub(r'[a-zA-Z]+', 'a', x)) #영어를 a로 바꿈.
  df['문장']=df['문장'].map(lambda x: re.sub(r'[^ a가-힣1]', '', x)) #한글, 1, a 제외 문자 제거

In [19]:
preprocess(df_train)

In [486]:
def random_deletion(words, p):
	
	new_words = []
	for token in words:
		r = random.uniform(0, 1)
		if token.text in stop_words_set and r>p: #1-p(0.5)의 확률로 불용어를 제거한다.
			continue
		if token.pos_ in ['ADJ', 'ADV', 'NUM'] and token.dep_ not in ['ROOT', 'fixed'] and r>p: #0.5확률로 수식하는 단어를 제거
			continue
		new_words.append(token.text) #포함되는 토큰을 담는다.

	return new_words

In [8]:
'때' in stop_words_set

True

In [ ]:
stop_words_set.remove('안')

In [238]:
neg_adv_lst=['못', '안', '전혀', '결코', '별로', '그리', '도대체', '도저히', '절대', '그다지', '절대로', '도무지', '채', '미처', '좀처럼', '여간', '영', '비단', '차마', '통', '구태여']

In [533]:
def random_deletion_neg(words, p):
	
	new_words = []
	for token in words:
		r = random.uniform(0, 1)
		if token.text in stop_words_set and r>p: #1-p(0.5)의 확률로 불용어를 제거한다.
			continue
		if token.pos_ in ['ADJ', 'NUM'] and token.dep_ != 'ROOT' and r>p: #0.5확률로 수식하는 단어를 제거
			continue
		if token.pos_ =='ADV' and token.text not in neg_adv_lst and r>p:
			continue
		new_words.append(token.text) #포함되는 토큰을 담는다.

	return new_words

In [547]:
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	if len(new_words) > 4:
		random_idx_1 = random.randint(1, len(new_words)-2)
		random_idx_2 = random_idx_1
		counter = 0

		while random_idx_2 == random_idx_1:
			random_idx_2 = random.randint(1, len(new_words)-2)
			counter += 1
			if counter > 3:
				return new_words

		new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


In [548]:
def EDA(sentence, alpha_rs=0.1, p_rd=0.5, num_aug=9):

	sentence = get_only_words(sentence)
	docs = nlp(sentence)

	augmented_sentences = []

	# rd
	for _ in range(num_aug):
		a_words = random_deletion(docs, p_rd)
		if len(a_words) > 4:
			augmented_sentences.append(" ".join(a_words))
	
	if len(augmented_sentences) < 2:
		augmented_sentences.append(sentence)

	
	# rs
	for i, sent in enumerate(augmented_sentences):
		words = sent.split(' ')
		words = [word for word in words if word != ""]
		if words:
			if len(words)<20:
				num_words=len(words)
				n_rs = max(1, int(alpha_rs*num_words))
				a_words = random_swap(words, n_rs)
				augmented_sentences[i] = " ".join(a_words)

	random.shuffle(augmented_sentences)


	return augmented_sentences

In [502]:
EDA('오늘 뭐 먹을래 김치만두?')

['오늘 먹을래 뭐 김치만두']

In [549]:
def EDA_neg(sentence, alpha_rs=0.1, p_rd=0.4, num_aug=9):

	sentence = get_only_words(sentence)
	docs = nlp(sentence)

	augmented_sentences = []

	# rd
	for _ in range(num_aug):
		a_words = random_deletion_neg(docs, p_rd)
		if len(a_words) > 4:
			augmented_sentences.append(" ".join(a_words))
	
	if len(augmented_sentences) < 2:
		augmented_sentences.append(sentence)

	
	# rs
	for i, sent in enumerate(augmented_sentences):
		words = sent.split(' ')
		words = [word for word in words if word != ""]
		if words:
			if len(words)<20:
				num_words=len(words)
				n_rs = max(1, int(alpha_rs*num_words))
				a_words = random_swap(words, n_rs)
				augmented_sentences[i] = " ".join(a_words)

	random.shuffle(augmented_sentences)


	return augmented_sentences

In [522]:
EDA('한번 꼭 따라 해보세요.')

['한번 따라 꼭 해보세요']

In [509]:
EDA_neg('나 그런 거 잘 못 하잖아')

['그런 잘 거 못 하잖아', '나 못 거 그런 하잖아', '그런 잘 거 못 하잖아']

In [550]:
def rs_rd_aug(df, label, category):
  aug_sent_lst=[]
  for sent in df['문장']:
    aug_sent_lst.extend(EDA(sent))

  aug_df={'증강문장_rs_rd':aug_sent_lst, label: [category]*len(aug_sent_lst)}

  return pd.DataFrame(aug_df)

In [551]:
def rs_rd_aug_neg(df, label, category):
  aug_sent_lst=[]
  for sent in df['문장']:
    aug_sent_lst.extend(EDA_neg(sent))

  aug_df={'증강문장_rs_rd':aug_sent_lst, label: [category]*len(aug_sent_lst)}

  return pd.DataFrame(aug_df)

In [554]:
df_train_true_aug = rs_rd_aug(df_train_true_aug, "유형", "사실형")
df_train_inf_aug = rs_rd_aug(df_train_inf_aug, "유형", "추론형")
df_train_pred_aug = rs_rd_aug(pred_aug, "유형", "예측형")
df_train_conv_aug = rs_rd_aug(df_train_conv_aug, "유형", "대화형")

df_train_pos_aug = rs_rd_aug(df_train_pos_aug, "극성", "긍정")
df_train_neg_aug = rs_rd_aug_neg(df_train_neg_aug, "극성", "부정")
df_train_non_aug = rs_rd_aug(non_aug, "극성", "미정")

df_train_pres_aug = rs_rd_aug(df_train_pres_aug, "시제", "현재")
df_train_past_aug = rs_rd_aug(df_train_past_aug, "시제", "과거")
df_train_future_aug = rs_rd_aug(df_train_future_aug, "시제", "미래")

df_train_certain_aug = rs_rd_aug(df_train_certain_aug, "확실성", "확실")
df_train_uncertain_aug = rs_rd_aug(df_train_uncertain_aug, "확실성", "불확실")

In [557]:
def label_concat(*args):
  df= args[0]
  for label in args[1:]:
    df=pd.concat([label, df], axis=0)

  return df

In [558]:
lst_kind=[df_train_true_aug, df_train_inf_aug, df_train_pred_aug, df_train_conv_aug]
lst_polarity=[df_train_pos_aug, df_train_neg_aug]
lst_tense=[df_train_pres_aug, df_train_past_aug, df_train_future_aug]
lst_certainty=[df_train_certain_aug, df_train_uncertain_aug]

In [559]:
df_kind_aug=label_concat(*lst_kind)

df_polar_aug=label_concat(*lst_polarity)

df_tense_aug=label_concat(*lst_tense)

df_certainty_aug=label_concat(*lst_certainty)

In [560]:
df_under_kind = df_train[df_train['유형']=='사실형'][['문장', '유형']].sample(frac=0.4)
df_under_kind=pd.concat([df_under_kind, df_train[df_train['유형']=='추론형'][['문장', '유형']]], axis=0)
df_kind_aug = df_kind_aug.rename(columns = {'증강문장_rs_rd': '문장'})
df_kind=pd.concat([df_under_kind, df_kind_aug], axis = 0)
df_kind.drop_duplicates(inplace=True)
df_kind.reset_index(drop=True, inplace=True)

df_under_polar = df_train[df_train['극성']=='긍정'][['문장', '극성']].sample(frac=0.4)
df_polar_aug = df_polar_aug.rename(columns = {'증강문장_rs_rd': '문장'})
df_polar = pd.concat([df_under_polar, df_polar_aug], axis = 0)
df_polar.drop_duplicates(inplace=True)
df_polar.reset_index(drop=True, inplace=True)

df_under_tense = df_train[df_train['시제']!='미래'][['문장', '시제']]
df_tense_aug = df_tense_aug.rename(columns = {'증강문장_rs_rd': '문장'})
df_tense = pd.concat([df_under_tense, df_tense_aug], axis = 0)
df_tense.drop_duplicates(inplace=True)
df_tense.reset_index(drop=True, inplace=True)

df_train_under = df_train[df_train['확실성']=='확실'][['문장', '확실성']].sample(frac=0.5)
df_certainty_aug = df_certainty_aug.rename(columns = {'증강문장_rs_rd': '문장'})
df_certainty = pd.concat([df_train_under, df_certainty_aug], axis = 0)
df_certainty.drop_duplicates(inplace=True)
df_certainty.reset_index(drop=True, inplace=True)

In [565]:
df_tense['시제'].value_counts()

미래    13593
과거     9599
현재     8347
Name: 시제, dtype: int64

In [568]:
df_kind

,문장,유형
0,박 회장은 1년 1월 경부고속도로 대전공구 시험계장으로 발령을 받는다,사실형
1,국내 게임업계에서 신종 코로나바이러스 감염증코로나1 환자가 발생했다,사실형
2,코로나1는 인류에게 많은 변화를 요구하고 있다,사실형
3,또 자유공정혁신연대를 1대 정책 기조로 잡고 경제운용 중심축을 정부에서 민간으로 바...,사실형
4,젊은 층 외지인 수요가 몰리면서 집값이 하늘을 모르고 치솟았다,사실형
...,...,...
18719,이날 한밤중인 한때 접속 대기자가 넘겼고 오전 오후 1만1여명이 몰렸다,사실형
18720,이날 오전 한때 넘겼고 대기자가 1만명을 접속 한밤중인 오후 1만1여명이 몰렸다,사실형
18721,이날 오전 오후 접속 대기자가 넘겼고 한밤중인 한때 1시에도 1만1여명이 몰렸다,사실형
18722,이날 오전 한때 접속 대기자가 1만명을 1만1여명이 한밤중인 오후 넘겼고 몰렸다,사실형


In [569]:
df_polar

,문장,극성
0,1세 여아가 숨진 사실을 경찰에 신고하기 하루 전인 지난해 1월 1일 김씨가 살던 ...,긍정
1,그는 국가를 대표해서 시나리오를 쓰는 건 아니지만 이건 한국에 가장 첫 오스카상이라...,긍정
2,심지어 조건도 더 좋다,긍정
3,기생충 책임 프로듀서로 전폭적인 지원을 해온 이 부회장은 나는 봉 감독의 모든 것을...,긍정
4,이외에도 넥슨은 메이플스토리 유니버스에서 획득한 a 기반으로 완전히 새로운 아이디어...,긍정
...,...,...
12167,전문가들은 연준이 자이언트 통해 금리 인상 가능성을 내달 시장의 불확실성을 해소하고...,긍정
12168,전문가들은 가능성을 자이언트 스탭과 내달 금리 인상 연준이 통해 시장의 불확실성을 ...,긍정
12169,전문가들은 연준이 자이언트 불확실성을 금리 인상 가능성을 통해 시장의 내달 해소하고...,긍정
12170,전문가들은 연준이 자이언트 신뢰를 내달 금리 인상 가능성을 통해 시장의 불확실성을 ...,긍정


In [570]:
df_certainty

,문장,확실성
0,특히 남자 옷은 더욱 그렇다,확실
1,농협은 a씨를 경찰에 신고했고 경찰은 전날 밤 1시 a씨를 긴급체포했다,확실
2,포식자 캐릭터는 기생충의 박 사장이 처음이 아니었다,확실
3,서울 소재 사립대 역시 이를 보전하려고 중국을 비롯한 해외 유학생의 지원금에 크게 ...,확실
4,하루라도 빨리 무관의 신인왕이란 허물을 벗어버리고 싶다,확실
...,...,...
20058,열린 a 투어 a 1타 차 단독 선두로 출발했지만 최종 라운드에서 무려 1타를 잃고...,확실
20059,열린 a 투어 a 한다 빅오픈에서 1타 단독 선두로 출발했지만 최종 라운드에서 무려...,확실
20060,앞서 열린 a 투어 a 한다 차 단독 선두로 출발했지만 최종 라운드에서 잃고 a 투...,확실
20061,앞서 열린 a 투어 a 단독 선두로 출발했지만 최종 라운드에서 무려 잃고 a 투어 ...,확실


In [571]:
df_tense

,문장,시제
0,11포인트 금리 인상은 1년 이후 1년 만에 처음이다,현재
1,이어 앞으로 전문가들과 함께 1주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...,과거
2,서울시는 올해 1월 즉시 견인 유예시간 1분을 제공하겠다고 밝혔지만 하루 만에 차도...,과거
3,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다,현재
4,이같은 변화를 포함해 올해 종부세 과세 대상은 당초 1만1명에서 1만1명으로 줄어든...,현재
...,...,...
31534,문화는 뿌리 없이 어느 어우러진 뚝 떨어진 것이 아니라 나라가 쌓아올린 다양한 산물...,현재
31535,문화는 뿌리 없이 어느 날 뚝 떨어진 것이 아니라 한 나라가 산물들이 다양한 쌓아올...,현재
31536,문화는 뿌리 없이 어느 날 열매이기 떨어진 것이 아니라 나라가 쌓아올린 다양한 산물...,현재
31537,문화는 뿌리 없이 날 뚝 산물들이 것이 아니라 한 나라가 쌓아올린 다양한 떨어진 어...,현재


In [ ]:
df_aug=df_kind.merge(df_polar, on=['문장'])

In [579]:
df_kind.to_csv('df_kind_aug.csv', index=False, encoding='utf-8-sig')
df_polar.to_csv('df_polar_aug.csv', index=False, encoding='utf-8-sig')
df_tense.to_csv('df_tense_aug.csv', index=False, encoding='utf-8-sig')
df_certainty.to_csv('df_certainty_aug.csv', index=False, encoding='utf-8-sig')